# 📥 COLETA GITLAB

Baixa documentos do GitLab para processamento.

In [9]:
import os
import sys
from pathlib import Path
from urllib.parse import urlparse

# Adicionar src ao path para usar nossa biblioteca customizada
sys.path.insert(0, str(Path().parent / "src"))
from gitlab_client import NicGitlab

# Configuração
GITLAB_URL = os.getenv("GITLAB_URL", "http://gitlab.processa.info/nic/documentacao/base-de-conhecimento.git")
GITLAB_TOKEN = os.getenv("GITLAB_ACCESS_TOKEN")
GITLAB_BRANCH = os.getenv("GITLAB_BRANCH", "main")
TARGET_FOLDER = os.getenv("GITLAB_TARGET_FOLDER", "30-Aprovados")
EXTENSIONS = [".pdf", ".docx", ".md", ".txt", ".png", ".jpg", ".jpeg"]

if not GITLAB_TOKEN:
    raise ValueError("GITLAB_ACCESS_TOKEN é obrigatório")

# Extrair informações da URL
clean_url = GITLAB_URL.removesuffix(".git")
parsed = urlparse(clean_url)
instance_url = f"{parsed.scheme}://{parsed.netloc}"
project_path = parsed.path.lstrip("/")
branch = GITLAB_BRANCH

print(f"GitLab: {instance_url}")
print(f"Projeto: {project_path}")
print(f"Branch: {branch}")
print(f"Pasta: {TARGET_FOLDER}")

GitLab: http://gitlab.processa.info
Projeto: nic/documentacao/base-de-conhecimento
Branch: main
Pasta: 30-Aprovados


In [10]:
# Conectar ao GitLab usando nosso cliente customizado
gl = NicGitlab(instance_url, private_token=GITLAB_TOKEN)
print("✅ GitLab conectado ok!")

# Obter projeto diretamente por path (sem busca de ID)
project = gl.client.get_project(project_path)
print(f"✅ Projeto do GitLab carregado ok!")
print(f"ID do projeto: {project.project_id}")

# Criar diretório de destino
docs_dir = Path("pipeline_data/documents")
docs_dir.mkdir(parents=True, exist_ok=True)

# Limpar diretório
print(f"Limpando diretório: {docs_dir}")
for f in docs_dir.glob("*"):
    if f.is_file():
        f.unlink()
print(f"✅ Diretório limpo!")

print(f"Diretório preparado: {docs_dir}")

✅ GitLab conectado ok!
✅ Projeto do GitLab carregado ok!
ID do projeto: 266
Limpando diretório: pipeline_data/documents
✅ Diretório limpo!
Diretório preparado: pipeline_data/documents


In [11]:
# Listar e filtrar arquivos
def list_files_recursive(project, path="", branch="main"):
    """Lista arquivos recursivamente em uma pasta"""
    files = []
    try:
        items = project.repository_tree(path=path, ref=branch, all=True)
        for item in items:
            if item["type"] == "blob":
                files.append({
                    "name": item["name"],
                    "path": item["path"],
                    "size": 0  # GitLab API não retorna tamanho na listagem
                })
    except Exception as e:
        print(f"Erro ao listar {path}: {e}")
    return files

# Listar arquivos na pasta alvo
print(f"Listando arquivos da pasta: {TARGET_FOLDER}")
all_files = list_files_recursive(project, TARGET_FOLDER)

# Filtrar por extensão
filtered_files = [
    {"name": i["name"], "path": i["path"], "size": 0}
    for i in project.repository_tree(path=TARGET_FOLDER, ref=branch, recursive=True, all=True, per_page=100)
    if i.get("type") == "blob" and any(i["name"].lower().endswith(ext.lower()) for ext in EXTENSIONS)
]

print(f"Arquivos encontrados: {len(all_files)}")
print(f"Arquivos filtrados: {len(filtered_files)}")

for f in filtered_files[:10]:  # Mostrar primeiros 10
    print(f"  {f['path']}")
if len(filtered_files) > 10:
    print(f"  ... e mais {len(filtered_files) - 10} arquivos")

Listando arquivos da pasta: 30-Aprovados


Arquivos encontrados: 1
Arquivos filtrados: 39
  30-Aprovados/Anexos/Imagens Self Checkout/FIgura 7 - Opção Menu.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 1 – Interface inicial do Self Checkout.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 2 – Interface de identificação do cliente.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 3 – Interface para registro do produto.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 4 - Msg produto na cesta.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 5 - Escolher a forma de pagamento.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 6 – Opção para Ajuda.jpg
  30-Aprovados/Anexos/Imagens Self Checkout/Figura 8 - Autenticacao do Menu.jpg
  30-Aprovados/Mapas/Processa Sistemas.md
  30-Aprovados/Mapas/Visão Geral do NIC.md
  ... e mais 29 arquivos


In [12]:
# Baixar arquivos preservando a estrutura de pastas
downloaded = 0
errors = []
total_size = 0

for file_info in filtered_files:
    try:
        # Obter conteúdo do arquivo usando nosso cliente customizado
        content = project.download_file_raw(file_path=file_info["path"], ref=branch)

        # Caminho local com subpastas reproduzidas
        local_path = docs_dir / Path(file_info["path"])
        local_path.parent.mkdir(parents=True, exist_ok=True)

        # Salvar
        with open(local_path, "wb") as f:
            f.write(content)

        file_size = len(content)
        total_size += file_size
        downloaded += 1

        print(f"✅ {file_info['path']} ({file_size/1024:.1f} KB)")

    except Exception as e:
        errors.append(f"{file_info['path']}: {str(e)}")
        print(f"❌ {file_info['path']}: {str(e)}")

print(f"\n📊 Resumo:")
print(f"  Baixados: {downloaded}")
print(f"  Erros: {len(errors)}")
print(f"  Tamanho total: {total_size/1024/1024:.1f} MB")

if errors:
    print("\n❌ Erros:")
    for error in errors[:5]:
        print(f"  {error}")

✅ 30-Aprovados/Anexos/Imagens Self Checkout/FIgura 7 - Opção Menu.jpg (26.6 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 1 – Interface inicial do Self Checkout.jpg (26.9 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 2 – Interface de identificação do cliente.jpg (25.3 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 3 – Interface para registro do produto.jpg (37.9 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 4 - Msg produto na cesta.jpg (23.4 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 5 - Escolher a forma de pagamento.jpg (32.6 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 6 – Opção para Ajuda.jpg (23.7 KB)
✅ 30-Aprovados/Anexos/Imagens Self Checkout/Figura 8 - Autenticacao do Menu.jpg (27.1 KB)
✅ 30-Aprovados/Mapas/Processa Sistemas.md (4.2 KB)
✅ 30-Aprovados/Mapas/Visão Geral do NIC.md (4.6 KB)
✅ 30-Aprovados/Mapas/Visão Geral do Self Checkout.md (14.4 KB)
✅ 30-Aprovados/Tópicos/Acesso ao menu do fiscal.md (3.5 KB)
✅ 30-Aprovados